<a href="https://colab.research.google.com/github/GabrielYYM/Notebooks/blob/main/AtividadeFinalSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descrição do Dataset

O presente dataset representa o perfil de microbiota intestinal de crianças com e sem Transtorno do Espectro Autista (ASD). Ele foi construído a partir de sequenciamento de 16S rRNA, gerando uma matriz de abundância categorizada de microrganismos.

    Amostras: 61 crianças

        Grupo A → Crianças com ASD (Autistic Spectrum Disorder)

        Grupo B → Crianças com desenvolvimento típico (TD – Typically Developing)

    Variáveis: 5619 microrganismos identificados em nível taxonômico (gênero e espécie), com nomes no formato padrão:

        Exemplo: g__Faecalibacterium;s__Faecalibacterium prausnitzii

    Natureza dos dados:

        Cada célula da matriz indica a categoria de abundância relativa de um microrganismo em uma amostra.

        Os valores são representados qualitativamente como low (baixa abundância), mid (abundância intermediária), high (alta abundância), além de casos específicos de absent (ausente) e present (presente).

Assim, o dataset reflete a composição microbiana intestinal de cada criança, permitindo investigar se há padrões que distinguem crianças com ASD das crianças com desenvolvimento típico.


🧩 Hipótese

A composição da microbiota intestinal difere significativamente entre crianças com Transtorno do Espectro Autista (ASD) e com desenvolvimento típico (TD), e tais diferenças podem ser detectadas e exploradas por métodos de aprendizado de máquina, como SVM, para discriminar os grupos com base no perfil microbiano.
❓ Questão norteadora

É possível identificar padrões na microbiota intestinal que discriminem crianças com ASD de crianças com desenvolvimento típico (TD) utilizando algoritmos de aprendizado de máquina, e como os diferentes métodos de busca de hiperparâmetros (GridSearchCV e RandomizedSearchCV) influenciam a performance dessa classificação?


📌 Atividade

    Exploração do Dataset

        Leia e descreva a composição do dataset (amostras, grupos e variáveis).

        Reflita sobre o que significa cada categoria de abundância (low, mid, high, absent, present).

    Experimento com GridSearchCV

        Utilize o método GridSearchCV, que faz uma busca exaustiva em todas as combinações de parâmetros.
        Reduzir o número de variáveis (Use.: k=500 ou k=100 com SelectKBest).

        Observe quais parâmetros foram escolhidos como ótimos e qual foi o desempenho do modelo (acurácia, sensibilidade, especificidade, ROC-AUC).

        Reflita: quais são as vantagens de testar todas as combinações possíveis? Quais são as limitações desse método?

    Experimento com RandomizedSearchCV

        Utilize o método RandomizedSearchCV, que faz uma busca aleatória em parte do espaço de parâmetros.

        Compare os resultados com os obtidos no GridSearchCV.

        Reflita: por que o RandomizedSearch pode encontrar resultados diferentes, às vezes melhores, às vezes piores, mesmo testando menos combinações?

    Análise crítica

        Compare os dois métodos considerando:

            Tempo de execução.

            Cobertura do espaço de parâmetros.

            Qualidade e estabilidade dos resultados.